# #0 Table of Content
1. Import packages and data
2. Data cleanse
3. r
4. r
5. f

# #1 Import packages, data and creating functions

In [3]:
import pandas as pd

In [4]:
review_table   = pd.read_csv("../asset/etlReview.csv")

In [5]:
review_table

,review_id,username,product_id,date,rating,content,location
0,6ef347c0-c603-422a-b8d1-c4b96bed0207,i*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,NaN,5,Best buy ever\r\nit looks great works great\r\...,2/9/2023 1:13
1,af3ccbed-1865-4492-88e3-723e9dda0de9,jessylim70,912d838b-5e84-4aef-a0dd-bb23f44e5913,NaN,5,Item received in good condition.\r\nBought dur...,21/7/2023 23:33
2,d17ebe72-919d-4c2a-a230-88119aac725c,s*****b,912d838b-5e84-4aef-a0dd-bb23f44e5913,NaN,5,Value For Money: yes\r\nBest Feature(s): comfo...,8/5/2023 19:37
3,9b72986c-7525-425e-89c6-e27d7e18af50,a*****w,912d838b-5e84-4aef-a0dd-bb23f44e5913,NaN,5,Impressed with the quality and looks really ni...,13/6/2023 11:03
4,3af89ba2-29c8-4adb-bb3a-cf9dd3ef2255,j*****n,912d838b-5e84-4aef-a0dd-bb23f44e5913,NaN,5,Value For Money: yes\r\nBest Feature(s): the w...,6/10/2023 12:56
...,...,...,...,...,...,...,...
137125,074c505d-d0c8-44c8-8c1c-3fa0bedf74bf,k88nc69dq9,46ec2b44-1271-425c-ae12-b43209d6442a,NaN,5,NaN,28/8/2021 6:27
137126,98bf5c9d-b634-4698-91e0-a7594486ba67,excessusfame,46ec2b44-1271-425c-ae12-b43209d6442a,NaN,5,NaN,8/8/2021 20:25
137127,9fdb76a8-f3f6-47dc-8f0b-9aaf57e02d23,medina0316,46ec2b44-1271-425c-ae12-b43209d6442a,NaN,5,NaN,23/7/2021 13:12
137128,e66b76c9-bee1-4b81-bcf5-35bae1ef061c,d8jwckvzlj,46ec2b44-1271-425c-ae12-b43209d6442a,NaN,5,NaN,19/6/2021 14:19
